# Backtester

%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Stock Predictor Lib


In [10]:
import json as js
import numpy as np
import pandas as pd
from ta import *
try:
    from fastai.imports import *
    from fastai.structured import *
    from fastai.column_data import *
except ImportError:
    print('*** Failed importing fast.ai modules ***')


class StockPredictor:

    def __init__(self, df, index):
        self.df = df
        self.index = index

    # ///////////////////////////////
    # /////// DATA CLEANING /////////
    # ///////////////////////////////

    def sample_train(self, sampleSize):
        self.train = self.df.iloc[-sampleSize:].copy()
        print('Train size: ' + str(len(self.train)) +
              ' Original size: ' + str(len(self.df)))

    def set_date_as_index(self):
        self.train.loc[:, self.index] = pd.to_datetime(
            self.train[self.index]).copy()

    def set_date_as_index_unix(self):
        self.train.loc[:, self.index] = pd.to_datetime(
            self.train[self.index], unit='s').copy()

    def split_test_train(self, percent_split):
        train_records_count = int(len(self.train) * percent_split)
        test_records_count = int(len(self.train) * (1-percent_split))
        self.test = self.train.tail(test_records_count)
        self.train = self.train.head(train_records_count)
        self.test = self.test.set_index('Timestamp')
        self.train = self.train.set_index('Timestamp')
        print('Train size: ' + str(len(self.train)) +
              ' Test size: ' + str(len(self.test)))

    def normalize_train(self, volume, open, high, low, close, weighted_price):
        self.train = pd.DataFrame({
            'Timestamp': self.train[self.index],
            'Volume': self.train[volume],
            'Open': self.train[open],
            'High': self.train[high],
            'Low': self.train[low],
            'Close': self.train[close],
            'Weighted_Price': self.train[weighted_price]
        })[['Timestamp', 'Volume', 'Open', 'High', 'Low', 'Close', 'Weighted_Price']]

    def clean_train(self):
        self.train = self.train.replace([np.inf, -np.inf], np.nan)
        self.train.fillna(method='ffill', inplace=True)
        self.train.fillna(method='bfill', inplace=True)
        print('Train size: ' + str(len(self.train)))

    """ Trim the beginning to have accurate TA values, trim the end to have accurate target value"""

    def trim_ends(self, begin_count, end_count):
        self.train = self.train.iloc[begin_count:]
        self.train = self.train.iloc[:-end_count]
        print('Trim beginning: ' + str(begin_count) + '. Trim end: ' +
              str(end_count) + '. Train size: ' + str(len(self.train)))

    # ///////////////////////////////
    # ////// FEATURE CREATION ///////
    # ///////////////////////////////

    def get_train_size(self, percent_split):
        train_records_count = int(len(self.train) * percent_split)
        print('Train size: ' + str(train_records_count) +
              ' percent_split: ' + str(percent_split))
        return train_records_count

    def get_validation_indexes(self, train_size, df):
        validation_indexes = list(range(train_size, len(df)))
        print('Validation Index size: ' + str(len(validation_indexes)))
        return validation_indexes

    def apply_variable_types(self, df, cat_vars, contin_vars, dep):
        dep_array = [] if dep is None else [dep]
        df = df[cat_vars + contin_vars + dep_array].copy()
        for v in cat_vars:
            df[v] = df[v].astype('category').cat.as_ordered()
        for v in contin_vars:
            df[v] = df[v].astype('float32')
        return df

    # ///////////////////////////////
    # //// FEATURE ENGINEERING //////
    # ///////////////////////////////

    def get_max_lookback(self, target, lookback):
        return self.train[target].rolling(window=lookback, min_periods=1).max()

    def get_min_lookback(self, target, lookback):
        return self.train[target].rolling(window=lookback, min_periods=1).min()

    def get_moving_average(self, target, lookback):
        return self.train[target].rolling(window=lookback, min_periods=1).mean()

    def get_max_lookahead(self, df, target, lookahead):
        return df[target] \
            .iloc[::-1] \
            .rolling(window=lookahead, min_periods=1) \
            .max() \
            .iloc[::-1]

    def get_last_lookahead(self, df, target, lookahead):
        return df[target].shift(-lookahead)

    def get_lookback(self, df, target, lookback):
        return df[target].shift(lookback)

    def add_historical_candles(self, df, lookback):
        for i in range(1, lookback):
            df[str(i) + 'Open'] = self.get_lookback(df,
                                                    'Open', i).astype('float')
            df[str(i) + 'High'] = self.get_lookback(df,
                                                    'High', i).astype('float')
            df[str(i) + 'Low'] = self.get_lookback(df,
                                                   'Low', i).astype('float')
            df[str(i) + 'Close'] = self.get_lookback(df,
                                                     'Close', i).astype('float')
            df[str(i) + 'Volume'] = self.get_lookback(df,
                                                      'Volume', i).astype('float')

    def add_ta(self):
        self.train = add_all_ta_features(
            self.train, "Open", "High", "Low", "Close", "Volume", fillna=True)
        self.train['sma5'] = self.get_moving_average("Close", 5)
        self.train['sma15'] = self.get_moving_average("Close", 15)
        self.train['sma30'] = self.get_moving_average("Close", 30)
        self.train['sma60'] = self.get_moving_average("Close", 60)
        self.train['sma90'] = self.get_moving_average("Close", 90)
        self.add_historical_candles(self.train, 30)
        # rsi = self.train['momentum_rsi']
        # self.train['rsi_category'] = rsi < 30

    """ Set the target (dependent variable) by looking ahead in a certain time window and percent increase
        to determine if the action should be a BUY or a SELL. BUY is true/1 SELL is false/0"""

    def set_target(self, target, lookahead, percentIncrease):
        max_lookahead = self.get_max_lookahead(self.train, target, lookahead)
        self.train['action'] = max_lookahead > (
            percentIncrease * self.train['Close'])
#        self.train['max'] =max_in_lookahead_timeframe
        self.train.action = self.train.action.astype(int)
        buy_count = str(len(self.train[self.train.action == 1]))
        sell_count = str(len(self.train[self.train.action == 0]))
        print('Buy count: ' + buy_count + ' Sell count: ' + sell_count)

    def set_target_hold(self, target, lookahead, percentIncrease):
        self.train['action'] = 0
        max_lookahead = self.get_max_lookahead(self.train, target, lookahead)
        self.train.loc[max_lookahead > self.train['Close'], 'action'] = 1
        self.train.loc[max_lookahead > percentIncrease *
                       self.train['Close'], 'action'] = 2
        self.train.action = self.train.action.astype(np.float32)
        sell_count = str(len(self.train[self.train.action == 0]))
        hold_count = str(len(self.train[self.train.action == 1]))
        buy_count = str(len(self.train[self.train.action == 2]))
        print('Buy count: ' + buy_count + ' Sell count: ' +
              sell_count + ' Hold count: ' + hold_count)

    def set_target_hold_reg(self, target, lookahead, percentIncrease):
        self.train['max_lookahead'] = 0
        max_lookahead = self.get_max_lookahead(self.train, target, lookahead)
        self.train['max_lookahead'] = max_lookahead
        self.train.max_lookahead = self.train.max_lookahead.astype(np.float32)

    def add_date_values(self):
        add_datepart(self.train, 'Timestamp', drop=False)
        self.train['hour'] = self.train['Timestamp'].dt.hour
        self.train['minute'] = self.train['Timestamp'].dt.minute

    def set_target_historical(self, target, lookback, percentIncrease):
        max_in_lookback_timeframe = self.get_max_lookback(target, lookback)
        self.train['action'] = max_in_lookback_timeframe > (
            percentIncrease * self.train['Close'])
        self.train.action = self.train.action.astype(int)
        buy_count = str(len(self.train[self.train.action == 1]))
        sell_count = str(len(self.train[self.train.action == 0]))
        print('Buy count: ' + buy_count + ' Sell count: ' + sell_count)

    def set_target_historical_hold(self, target, lookback, percentIncrease):
        self.train['action'] = 0
        max_lookback = self.get_max_lookback(target, lookback)
        self.train.loc[max_lookback > self.train['Close'], 'action'] = 1

        self.train.loc[max_lookback > percentIncrease *
                       self.train['Close'], 'action'] = 2

        self.train.action = self.train.action.astype(int)
        sell_count = str(len(self.train[self.train.action == 0]))
        hold_count = str(len(self.train[self.train.action == 1]))
        buy_count = str(len(self.train[self.train.action == 2]))
        print('Buy count: ' + buy_count + ' Sell count: ' +
              sell_count + ' Hold count: ' + hold_count)

    # ///////////////////////////////
    # ///////// EVALUATION //////////
    # ///////////////////////////////

    def generate_net_profit_result(self, df, startAmount, totalBuys, totalSells):
        startClose = df.Close.iloc[0]
        endClose = df.Close.iloc[-1]
        endBuyAmount = df.buyAmount.iloc[-1]
        endSellAmount = df.sellAmount.iloc[-1]
        endAmount = endBuyAmount if (
            endBuyAmount > 0) else (endSellAmount * endClose)
        buyAndHoldPercentIncrease = ((endClose - startClose)/startClose) * 100
        percentIncrease = ((endAmount - startAmount)/startAmount) * 100
        percentDifference = percentIncrease - buyAndHoldPercentIncrease

        result = {
            'startClose': startClose,
            'endClose': endClose,
            'startAmount': startAmount,
            'endAmount': endAmount,
            'buyAndHoldPercentIncrease': round(buyAndHoldPercentIncrease, 3),
            'percentIncrease': round(percentIncrease, 3),
            'percentDifference': round(percentDifference, 3),
            'totalTrades': totalBuys + totalSells
        }
        return result

    def calculate_accuracy_hold(self, df):
        successful_predictions = df.loc[df.action == df.predicted]
        total_accuracy = len(successful_predictions)/len(df)
        total_sell_actions = df.loc[df.action == 0]
        total_hold_actions = df.loc[df.action == 1]
        total_buy_actions = df.loc[df.action == 2]
        successful_sell_predictions = successful_predictions.loc[successful_predictions.action == 0]
        successful_hold_predictions = successful_predictions.loc[successful_predictions.action == 1]
        successful_buy_predictions = successful_predictions.loc[successful_predictions.action == 2]
        sell_accuracy = len(successful_sell_predictions) / \
            len(total_sell_actions)
        hold_accuracy = len(successful_hold_predictions) / \
            len(total_hold_actions)
        buy_accuracy = len(successful_buy_predictions)/len(total_buy_actions)
        f1Score = (buy_accuracy + sell_accuracy + hold_accuracy)/3
        result = {
            'F1Score': round(f1Score, 3),
            'totalAccuracy': round(total_accuracy, 3),
            'buyAccuracy': round(buy_accuracy, 3),
            'sellAccuracy': round(sell_accuracy, 3),
            'hold_accuracy': round(hold_accuracy, 3),
            'totalSellActions': len(total_sell_actions),
            'totalHoldActions': len(total_hold_actions),
            'totalBuyActions': len(total_buy_actions),
            'successfulBuyPredictions': len(successful_buy_predictions)
        }
        return result

    def calculate_accuracy(self, df):
        successful_predictions = df.loc[df.action == df.predicted]
        total_accuracy = len(successful_predictions)/len(df)
        total_buy_actions = df.loc[df.action == 1]
        total_sell_actions = df.loc[df.action == 0]
        successful_buy_predictions = successful_predictions.loc[successful_predictions.action == 1]
        successful_sell_predictions = successful_predictions.loc[successful_predictions.action == 0]
        buy_accuracy = len(successful_buy_predictions)/len(total_buy_actions)
        sell_accuracy = (len(successful_sell_predictions) /
                         len(total_sell_actions))
        f1Score = (buy_accuracy + sell_accuracy)/2
        result = {
            'F1Score': round(f1Score, 3),
            'totalAccuracy': round(total_accuracy, 3),
            'buyAccuracy': round(buy_accuracy, 3),
            'sellAccuracy': round(sell_accuracy, 3),
            'totalBuyActions': len(total_buy_actions),
            'successfulBuyPredictions': len(successful_buy_predictions)
        }
        return result

    def calculate_net_profit(self, inputDf, startAmount, fee):
        df = inputDf
        df['buyAmount'] = 0
        df['sellAmount'] = 0
        totalBuys = 0
        totalSells = 0
        for index, row in df.iterrows():
            prevBuyAmount = df.buyAmount.get(index - 1, np.nan)
            prevSellAmount = df.sellAmount.get(index - 1, np.nan)
            predicted = row.predicted
            if index == df.index[0]:
                df.loc[index, 'buyAmount'] = startAmount
            elif predicted == 1 and prevBuyAmount > 0:
                # BUY
                base_sell = prevBuyAmount/row.Close
                df.loc[index, 'sellAmount'] = base_sell - (base_sell * fee)
                totalBuys += 1
            elif predicted == 1 and prevBuyAmount == 0:
                df.loc[index, 'sellAmount'] = prevSellAmount
            elif predicted == 0 and prevSellAmount > 0:
                # SELL
                base_buy = prevSellAmount*row.Close
                df.loc[index, 'buyAmount'] = base_buy - (base_buy*fee)
                totalSells += 1
            elif predicted == 0 and prevSellAmount == 0:
                df.loc[index, 'buyAmount'] = prevBuyAmount
            else:
                raise ValueError(
                    'This is weird, invalid predicted value: ' + str(predicted) + ' prevSellAmount: ' +
                    str(prevSellAmount) + ' prevBuyAmount: ' + str(prevBuyAmount))
        result = self.generate_net_profit_result(
            df, startAmount, totalBuys, totalSells)
        self.net_profit_df = df
        self.result = result
        # print(js.dumps(result, sort_keys=False, indent=4, separators=(',', ': ')))

    def calculate_net_profit_hold(self, inputDf, startAmount, fee):
        df = inputDf
        df['buyAmount'] = 0
        df['sellAmount'] = 0
        totalBuys = 0
        totalSells = 0
        for index, row in df.iterrows():
            prevBuyAmount = df.buyAmount.get(index - 1, np.nan)
            prevSellAmount = df.sellAmount.get(index - 1, np.nan)
            predicted = row.predicted
            if index == df.index[0]:
                df.loc[index, 'buyAmount'] = startAmount
            elif predicted == 2 and prevBuyAmount > 0:
                # BUY
                base_sell = prevBuyAmount / row.Close
                df.loc[index, 'sellAmount'] = base_sell - (base_sell * fee)
                totalBuys += 1
            elif predicted == 2 and prevBuyAmount == 0:
                df.loc[index, 'sellAmount'] = prevSellAmount
            elif predicted == 0 and prevSellAmount > 0:
                # SELL
                base_buy = prevSellAmount * row.Close
                df.loc[index, 'buyAmount'] = base_buy - (base_buy*fee)
                totalSells += 1
            elif predicted == 0 and prevSellAmount == 0:
                df.loc[index, 'buyAmount'] = prevBuyAmount
            elif predicted == 1:
                # HOLD
                df.loc[index, 'buyAmount'] = prevBuyAmount
                df.loc[index, 'sellAmount'] = prevSellAmount
            else:
                raise ValueError(
                    'This is weird, invalid predicted value: ' + str(predicted))

        result = self.generate_net_profit_result(
            df, startAmount, totalBuys, totalSells)
        self.net_profit_df = df
        self.result = result
        # print(js.dumps(result, sort_keys=False, indent=4, separators=(',', ': ')))

    def calculate_net_profit_hold_reg(self, inputDf, startAmount, fee, percentIncrease):
        df = inputDf
        df['buyAmount'] = 0
        df['sellAmount'] = 0
        totalBuys = 0
        totalSells = 0
        for index, row in df.iterrows():
            prevBuyAmount = df.buyAmount.get(index - 1, np.nan)
            prevSellAmount = df.sellAmount.get(index - 1, np.nan)
            predicted = row.predicted
            threshold = percentIncrease * row.Close
            if index == df.index[0]:
                df.loc[index, 'buyAmount'] = startAmount
            elif predicted >= threshold and prevBuyAmount > 0:
                # BUY
                base_sell = prevBuyAmount / row.Close
                df.loc[index, 'sellAmount'] = base_sell - (base_sell * fee)
                totalBuys += 1
            elif predicted >= threshold and prevBuyAmount == 0:
                df.loc[index, 'sellAmount'] = prevSellAmount
            elif predicted >= row.Close:
                # HOLD
                df.loc[index, 'buyAmount'] = prevBuyAmount
                df.loc[index, 'sellAmount'] = prevSellAmount
            elif predicted < row.Close and prevSellAmount > 0:
                # SELL
                base_buy = prevSellAmount * row.Close
                df.loc[index, 'buyAmount'] = base_buy - (base_buy*fee)
                totalSells += 1
            elif predicted < row.Close and prevSellAmount == 0:
                df.loc[index, 'buyAmount'] = prevBuyAmount
            else:
                raise ValueError(
                    'This is weird, invalid predicted value: ' + str(predicted) + ' Close: ' + str(row.Close))

        result = self.generate_net_profit_result(
            df, startAmount, totalBuys, totalSells)
        self.net_profit_df = df
        self.result = result

    # ///////////////////////////////
    # /////////// UTIL //////////////
    # ///////////////////////////////

    def save_to_feather(self, path):
        self.train.reset_index(inplace=True)
        self.train.to_feather(f'{PATH}train')

    def read_from_feather(self, PATH):
        self.train = pd.read_feather(f'{PATH}train')
        # train.drop(self.index,1,inplace=True)

    """ usage conflateTimeFrame(df, '5T') """

    def conflate_time_frame(self, df, timeFrame):
        ohlc_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }
        return df.resample(timeFrame).agg(ohlc_dict)

    def plot_profit(self, df):
        df.reset_index(inplace=True)
        df.plot(
            x='Timestamp',
            y=['Close', 'buyAmount'],
            style='o',
            figsize=(10, 5),
            grid=True)


SyntaxError: invalid syntax (<ipython-input-10-f55f65690f9f>, line 405)

In [5]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
        if right_on is None:
            right_on = left_on
        return left.merge(right, how='left', left_on=left_on, right_on=right_on,
                          suffixes=("", suffix))

## Config


In [6]:
np.set_printoptions(threshold=50, edgeitems=20)
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
from IPython.display import HTML

In [7]:
lookahead = 8
sample_size=300000
percentIncrease = 1.002
index='Timestamp'
# index='time_period_start'
dep = 'action'
PATH='../data/stock/'

## Create datasets

In [8]:
table_names = [
#         'btc-bitstamp-2012-01-01_to_2018-01-08'
#     'BTC_COINBASE_2018-07-25_09-06'
#     'ETH_COINBASE_07-21_08-24'
#     'COINBASE_BCH_2018-06-15_09-01'
#     'COINBASE_BTC_2017-11-01_01-09'
#     'bitstampUSD_1-min_data_2012-01-01_to_2018-06-27',
    'coinbaseUSD_1-min_data_2014-12-01_to_2018-06-27'
#     'bitstamp_07-09'
]

In [9]:
tables = [pd.read_csv(f'{PATH}{fname}.csv', low_memory=False) for fname in table_names]

SyntaxError: invalid syntax (<ipython-input-9-88c571127c2d>, line 1)

In [ ]:
# for t in tables: display(t.head())

In [ ]:
train= tables[0]

In [ ]:
p = StockPredictor(train, index)
p.sample_train(sample_size)
# p.train = p.train.head(100000)

p.normalize_train('Volume_(BTC)','Open','High','Low','Close', 'Weighted_Price')
last_df = p.train.tail(100)
last_df

## Convert to JSON

In [ ]:
train_json = last_df.to_json(orient='records')

In [ ]:
# train_json

In [ ]:
converted_df = pd.read_json(train_json)
# converted_df

In [ ]:
body = str(train_json)
# print("body: {}".format(body))
# body

In [ ]:
hist_str = '[{"Timestamp":1530051660,"Volume":20.11331257,"Open":6132.99,"High":6140.0,"Low":6132.99,"Close":6140.0,"Weighted_Price":6135.5774024},{"Timestamp":1530051720,"Volume":5.35401256,"Open":6140.0,"High":6149.99,"Low":6140.0,"Close":6148.07,"Weighted_Price":6144.8826792},{"Timestamp":1530051780,"Volume":3.4439908,"Open":6148.11,"High":6148.11,"Low":6139.04,"Close":6143.35,"Weighted_Price":6143.5173427},{"Timestamp":1530051840,"Volume":4.97508775,"Open":6143.33,"High":6150.0,"Low":6143.33,"Close":6150.0,"Weighted_Price":6145.3558679},{"Timestamp":1530051900,"Volume":6.99843103,"Open":6149.99,"High":6149.99,"Low":6143.12,"Close":6145.67,"Weighted_Price":6144.4568306},{"Timestamp":1530051960,"Volume":7.31167025,"Open":6145.67,"High":6162.49,"Low":6145.66,"Close":6162.49,"Weighted_Price":6153.3112079},{"Timestamp":1530052020,"Volume":17.14013855,"Open":6162.49,"High":6165.0,"Low":6156.0,"Close":6156.0,"Weighted_Price":6161.7918629},{"Timestamp":1530052080,"Volume":7.59042159,"Open":6156.4,"High":6156.4,"Low":6148.31,"Close":6150.03,"Weighted_Price":6154.9712096},{"Timestamp":1530052140,"Volume":1.4225527,"Open":6150.02,"High":6150.02,"Low":6145.66,"Close":6145.67,"Weighted_Price":6146.3879528},{"Timestamp":1530052200,"Volume":3.15632473,"Open":6145.67,"High":6145.67,"Low":6145.66,"Close":6145.67,"Weighted_Price":6145.6658289},{"Timestamp":1530052260,"Volume":4.78356474,"Open":6145.67,"High":6160.0,"Low":6145.67,"Close":6160.0,"Weighted_Price":6153.8187541},{"Timestamp":1530052320,"Volume":2.14995586,"Open":6160.0,"High":6160.0,"Low":6151.68,"Close":6151.68,"Weighted_Price":6154.2346645},{"Timestamp":1530052380,"Volume":3.27437845,"Open":6151.26,"High":6153.0,"Low":6145.66,"Close":6152.97,"Weighted_Price":6148.9664802},{"Timestamp":1530052440,"Volume":1.1283,"Open":6152.97,"High":6153.0,"Low":6152.96,"Close":6153.0,"Weighted_Price":6152.9636905},{"Timestamp":1530052500,"Volume":4.46873007,"Open":6153.0,"High":6154.98,"Low":6149.61,"Close":6149.61,"Weighted_Price":6153.4027893},{"Timestamp":1530052560,"Volume":3.09175356,"Open":6149.6,"High":6153.0,"Low":6149.6,"Close":6149.61,"Weighted_Price":6150.7242356},{"Timestamp":1530052620,"Volume":0.93808115,"Open":6149.61,"High":6149.61,"Low":6146.29,"Close":6146.29,"Weighted_Price":6147.4188589},{"Timestamp":1530052680,"Volume":12.13810613,"Open":6146.3,"High":6152.76,"Low":6141.04,"Close":6144.61,"Weighted_Price":6146.7642588},{"Timestamp":1530052740,"Volume":1.37074718,"Open":6141.04,"High":6153.12,"Low":6140.0,"Close":6153.12,"Weighted_Price":6143.1178694},{"Timestamp":1530052800,"Volume":1.11315542,"Open":6153.12,"High":6154.16,"Low":6145.61,"Close":6151.91,"Weighted_Price":6150.5199622},{"Timestamp":1530052860,"Volume":3.56171842,"Open":6149.99,"High":6149.99,"Low":6141.18,"Close":6141.19,"Weighted_Price":6147.0575467},{"Timestamp":1530052920,"Volume":0.9496159,"Open":6141.19,"High":6149.16,"Low":6141.19,"Close":6149.15,"Weighted_Price":6146.9956812},{"Timestamp":1530052980,"Volume":0.88176378,"Open":6149.15,"High":6151.95,"Low":6149.15,"Close":6151.94,"Weighted_Price":6150.4434109},{"Timestamp":1530053040,"Volume":7.98305488,"Open":6151.94,"High":6154.0,"Low":6151.94,"Close":6153.99,"Weighted_Price":6153.9639453},{"Timestamp":1530053100,"Volume":4.71993739,"Open":6153.99,"High":6155.0,"Low":6153.99,"Close":6155.0,"Weighted_Price":6154.5438028},{"Timestamp":1530053160,"Volume":2.38023359,"Open":6155.0,"High":6160.0,"Low":6154.99,"Close":6160.0,"Weighted_Price":6157.0548222},{"Timestamp":1530053220,"Volume":1.92970631,"Open":6159.99,"High":6160.0,"Low":6159.99,"Close":6159.99,"Weighted_Price":6159.9965501},{"Timestamp":1530053280,"Volume":0.8414,"Open":6160.0,"High":6160.0,"Low":6159.99,"Close":6159.99,"Weighted_Price":6159.9986106},{"Timestamp":1530053340,"Volume":0.61280969,"Open":6160.0,"High":6160.0,"Low":6159.99,"Close":6159.99,"Weighted_Price":6159.995767},{"Timestamp":1530053400,"Volume":1.09813591,"Open":6160.0,"High":6160.0,"Low":6159.99,"Close":6159.99,"Weighted_Price":6159.9933648},{"Timestamp":1530053460,"Volume":1.959,"Open":6159.99,"High":6159.99,"Low":6158.0,"Close":6158.0,"Weighted_Price":6159.0455239},{"Timestamp":1530053520,"Volume":2.46212921,"Open":6158.01,"High":6160.0,"Low":6158.01,"Close":6160.0,"Weighted_Price":6159.898143},{"Timestamp":1530053580,"Volume":1.23192567,"Open":6159.99,"High":6160.0,"Low":6159.99,"Close":6159.99,"Weighted_Price":6159.9939891},{"Timestamp":1530053640,"Volume":1.16608251,"Open":6160.0,"High":6160.0,"Low":6152.54,"Close":6152.54,"Weighted_Price":6158.0253904},{"Timestamp":1530053700,"Volume":1.64127442,"Open":6152.55,"High":6159.75,"Low":6152.55,"Close":6159.74,"Weighted_Price":6159.4213126},{"Timestamp":1530053760,"Volume":0.5529,"Open":6156.56,"High":6156.57,"Low":6156.56,"Close":6156.56,"Weighted_Price":6156.560331},{"Timestamp":1530053820,"Volume":1.67816614,"Open":6156.56,"High":6156.57,"Low":6156.56,"Close":6156.56,"Weighted_Price":6156.5672136},{"Timestamp":1530053880,"Volume":1.09093884,"Open":6156.56,"High":6156.57,"Low":6156.56,"Close":6156.56,"Weighted_Price":6156.5600297},{"Timestamp":1530053940,"Volume":1.17646717,"Open":6156.56,"High":6156.57,"Low":6156.56,"Close":6156.57,"Weighted_Price":6156.5606436},{"Timestamp":1530054000,"Volume":2.27768807,"Open":6156.57,"High":6156.57,"Low":6152.55,"Close":6152.55,"Weighted_Price":6155.815861},{"Timestamp":1530054060,"Volume":1.84941684,"Open":6152.55,"High":6152.56,"Low":6148.01,"Close":6148.02,"Weighted_Price":6151.5227531},{"Timestamp":1530054120,"Volume":7.99939416,"Open":6148.01,"High":6148.02,"Low":6148.01,"Close":6148.02,"Weighted_Price":6148.0140652},{"Timestamp":1530054180,"Volume":1.98125256,"Open":6148.01,"High":6148.02,"Low":6148.01,"Close":6148.02,"Weighted_Price":6148.0164409},{"Timestamp":1530054240,"Volume":1.23516913,"Open":6148.02,"High":6148.02,"Low":6148.01,"Close":6148.02,"Weighted_Price":6148.0184051},{"Timestamp":1530054300,"Volume":0.8904,"Open":6148.01,"High":6148.02,"Low":6148.01,"Close":6148.02,"Weighted_Price":6148.0156559},{"Timestamp":1530054360,"Volume":1.22594998,"Open":6148.02,"High":6148.02,"Low":6148.01,"Close":6148.02,"Weighted_Price":6148.0114927},{"Timestamp":1530054420,"Volume":19.49083657,"Open":6148.01,"High":6148.02,"Low":6140.0,"Close":6140.0,"Weighted_Price":6144.4470644},{"Timestamp":1530054480,"Volume":8.99812576,"Open":6140.0,"High":6140.0,"Low":6131.99,"Close":6139.39,"Weighted_Price":6136.4023309},{"Timestamp":1530054540,"Volume":7.29152073,"Open":6139.38,"High":6139.38,"Low":6131.99,"Close":6131.99,"Weighted_Price":6135.6686571},{"Timestamp":1530054600,"Volume":21.08649108,"Open":6131.99,"High":6131.99,"Low":6108.55,"Close":6123.88,"Weighted_Price":6122.3954496},{"Timestamp":1530054660,"Volume":10.80819557,"Open":6123.88,"High":6130.33,"Low":6123.88,"Close":6130.33,"Weighted_Price":6128.159472},{"Timestamp":1530054720,"Volume":12.82021312,"Open":6130.33,"High":6137.28,"Low":6129.32,"Close":6129.33,"Weighted_Price":6132.5454171},{"Timestamp":1530054780,"Volume":8.12705729,"Open":6129.33,"High":6137.89,"Low":6129.33,"Close":6134.51,"Weighted_Price":6134.2583158},{"Timestamp":1530054840,"Volume":32.4607466,"Open":6134.51,"High":6134.52,"Low":6118.76,"Close":6118.76,"Weighted_Price":6134.0405544},{"Timestamp":1530054900,"Volume":3.46303054,"Open":6118.75,"High":6133.03,"Low":6118.75,"Close":6133.03,"Weighted_Price":6123.5979427},{"Timestamp":1530054960,"Volume":4.50708598,"Open":6133.03,"High":6133.03,"Low":6128.75,"Close":6128.75,"Weighted_Price":6132.9226701},{"Timestamp":1530055020,"Volume":0.61007758,"Open":6130.65,"High":6130.65,"Low":6125.82,"Close":6125.82,"Weighted_Price":6126.0964874},{"Timestamp":1530055080,"Volume":14.30758,"Open":6125.82,"High":6133.66,"Low":6125.82,"Close":6133.65,"Weighted_Price":6129.4966658},{"Timestamp":1530055140,"Volume":0.96336806,"Open":6133.65,"High":6135.84,"Low":6133.65,"Close":6135.84,"Weighted_Price":6134.6099789},{"Timestamp":1530055200,"Volume":1.34355797,"Open":6135.83,"High":6136.0,"Low":6135.83,"Close":6136.0,"Weighted_Price":6135.8662292},{"Timestamp":1530055260,"Volume":3.42510815,"Open":6135.99,"High":6136.0,"Low":6135.99,"Close":6136.0,"Weighted_Price":6135.990806},{"Timestamp":1530055320,"Volume":0.25494551,"Open":6136.0,"High":6136.0,"Low":6135.99,"Close":6136.0,"Weighted_Price":6135.9965797},{"Timestamp":1530055380,"Volume":2.8049015,"Open":6136.0,"High":6136.0,"Low":6135.83,"Close":6135.83,"Weighted_Price":6135.9295228},{"Timestamp":1530055440,"Volume":0.7989,"Open":6135.83,"High":6135.84,"Low":6135.83,"Close":6135.83,"Weighted_Price":6135.8323069},{"Timestamp":1530055500,"Volume":0.3011,"Open":6135.83,"High":6135.84,"Low":6135.83,"Close":6135.84,"Weighted_Price":6135.8317038},{"Timestamp":1530055560,"Volume":7.4564,"Open":6135.84,"High":6135.84,"Low":6135.83,"Close":6135.84,"Weighted_Price":6135.8342389},{"Timestamp":1530055620,"Volume":0.83032178,"Open":6135.84,"High":6135.84,"Low":6135.83,"Close":6135.83,"Weighted_Price":6135.8349186},{"Timestamp":1530055680,"Volume":4.13413217,"Open":6135.83,"High":6135.83,"Low":6132.95,"Close":6135.01,"Weighted_Price":6134.7750378},{"Timestamp":1530055740,"Volume":14.80428443,"Open":6135.0,"High":6139.99,"Low":6135.0,"Close":6139.99,"Weighted_Price":6135.4244954},{"Timestamp":1530055800,"Volume":4.58328198,"Open":6140.0,"High":6140.0,"Low":6139.99,"Close":6139.99,"Weighted_Price":6139.9986293},{"Timestamp":1530055860,"Volume":3.91212059,"Open":6138.05,"High":6138.05,"Low":6131.0,"Close":6131.0,"Weighted_Price":6134.0308306},{"Timestamp":1530055920,"Volume":38.62488315,"Open":6131.01,"High":6131.01,"Low":6120.84,"Close":6120.84,"Weighted_Price":6130.1284859},{"Timestamp":1530055980,"Volume":26.35372511,"Open":6120.83,"High":6120.83,"Low":6119.13,"Close":6119.13,"Weighted_Price":6120.2228378},{"Timestamp":1530056040,"Volume":18.11776783,"Open":6119.13,"High":6119.14,"Low":6105.52,"Close":6105.52,"Weighted_Price":6116.8661471},{"Timestamp":1530056100,"Volume":22.4432696,"Open":6105.52,"High":6107.02,"Low":6100.35,"Close":6100.35,"Weighted_Price":6103.1611926},{"Timestamp":1530056160,"Volume":12.53263092,"Open":6100.01,"High":6104.19,"Low":6092.11,"Close":6103.48,"Weighted_Price":6096.0597592},{"Timestamp":1530056220,"Volume":18.79039921,"Open":6103.47,"High":6119.13,"Low":6095.11,"Close":6109.01,"Weighted_Price":6110.2696406},{"Timestamp":1530056280,"Volume":7.63187369,"Open":6109.02,"High":6109.02,"Low":6100.0,"Close":6100.01,"Weighted_Price":6104.244173},{"Timestamp":1530056340,"Volume":7.2106425,"Open":6100.0,"High":6100.01,"Low":6096.19,"Close":6096.19,"Weighted_Price":6099.9705253},{"Timestamp":1530056400,"Volume":4.11712665,"Open":6095.35,"High":6111.67,"Low":6095.35,"Close":6111.67,"Weighted_Price":6101.846787},{"Timestamp":1530056460,"Volume":3.66652478,"Open":6111.66,"High":6111.67,"Low":6107.53,"Close":6109.99,"Weighted_Price":6110.3003049},{"Timestamp":1530056520,"Volume":3.92956197,"Open":6109.99,"High":6110.0,"Low":6101.48,"Close":6103.7,"Weighted_Price":6107.0505684},{"Timestamp":1530056580,"Volume":32.7787774,"Open":6103.56,"High":6103.57,"Low":6089.04,"Close":6097.05,"Weighted_Price":6093.1476501},{"Timestamp":1530056640,"Volume":2.42002986,"Open":6103.32,"High":6103.58,"Low":6103.32,"Close":6103.58,"Weighted_Price":6103.5719767},{"Timestamp":1530056700,"Volume":4.06670597,"Open":6103.58,"High":6103.58,"Low":6091.0,"Close":6099.46,"Weighted_Price":6097.9082759},{"Timestamp":1530056760,"Volume":328.44281208,"Open":6099.47,"High":6099.47,"Low":6006.39,"Close":6049.54,"Weighted_Price":6051.0781956},{"Timestamp":1530056820,"Volume":96.39086956,"Open":6049.54,"High":6068.25,"Low":6048.0,"Close":6068.24,"Weighted_Price":6055.9871537},{"Timestamp":1530056880,"Volume":14.08567455,"Open":6068.25,"High":6079.99,"Low":6066.81,"Close":6066.81,"Weighted_Price":6073.797829},{"Timestamp":1530056940,"Volume":16.69867296,"Open":6066.09,"High":6066.1,"Low":6048.0,"Close":6048.01,"Weighted_Price":6054.4124581},{"Timestamp":1530057000,"Volume":8.96876625,"Open":6051.01,"High":6065.99,"Low":6051.01,"Close":6059.99,"Weighted_Price":6055.7096742},{"Timestamp":1530057060,"Volume":7.22665869,"Open":6060.0,"High":6065.99,"Low":6059.99,"Close":6065.99,"Weighted_Price":6061.3152763},{"Timestamp":1530057120,"Volume":29.52612135,"Open":6065.99,"High":6079.06,"Low":6065.98,"Close":6078.06,"Weighted_Price":6067.0461987},{"Timestamp":1530057180,"Volume":17.58497978,"Open":6078.06,"High":6078.06,"Low":6060.0,"Close":6065.98,"Weighted_Price":6069.1856369},{"Timestamp":1530057240,"Volume":1.24434387,"Open":6065.98,"High":6070.15,"Low":6065.0,"Close":6065.0,"Weighted_Price":6067.7212989},{"Timestamp":1530057300,"Volume":2.5166374,"Open":6065.0,"High":6070.16,"Low":6060.01,"Close":6064.02,"Weighted_Price":6065.0274307},{"Timestamp":1530057360,"Volume":10.56197347,"Open":6064.03,"High":6080.0,"Low":6064.03,"Close":6080.0,"Weighted_Price":6073.5649901},{"Timestamp":1530057420,"Volume":0.68200147,"Open":6080.0,"High":6080.0,"Low":6071.52,"Close":6077.99,"Weighted_Price":6078.136687},{"Timestamp":1530057480,"Volume":5.71018466,"Open":6078.0,"High":6081.0,"Low":6077.99,"Close":6081.0,"Weighted_Price":6078.3725148},{"Timestamp":1530057540,"Volume":20.89265755,"Open":6081.0,"High":6085.7,"Low":6074.0,"Close":6074.0,"Weighted_Price":6081.1052712},{"Timestamp":1530057600,"Volume":33.89811156,"Open":6074.01,"High":6074.01,"Low":6071.52,"Close":6071.53,"Weighted_Price":6073.9699599}]'
hist_json = js.loads(hist_str)

In [ ]:
# hist_json

In [ ]:
hist_df = pd.read_json(hist_str, orient='list')

In [ ]:
# hist_df